# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/golden/Documents/my

Import the object detection module.

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [3]:
#Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/golden/Documents/VOCdevkit/output/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/golden/Documents/VOCdevkit/label.pbtxt'

NUM_CLASSES = 1

In [4]:
#Load a (frozen) Tensorflow model into memory.

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


In [5]:

#Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

++++DBUG Label Map: 


In [6]:
#
#Helper code
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [7]:


# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/golden/Documents/VOCdevkit/test/maq/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'testIso_{}.jpg'.format(i)) for i in range(12, 13) ]
0
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)



In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      #print(boxes)
      #print(scores)
      #print(classes)
      #print(num)
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      #plt.figure(figsize=IMAGE_SIZE)
      plt.figure(figsize=(15, 7))
      plt.imshow(image_np)
      t=np.shape(image_np)
      img2 = np.zeros((1000,2000,3))

      for i in range(int(t[1]*boxes[0][0][1]),int(t[1]*boxes[0][0][3])):
        
        for j in range(int(t[0]*boxes[0][0][0]),int(t[0]*boxes[0][0][2])):
            
        
            #img2[j][i][0]=image_np[j][i][0]
            #img2[j][i][1]=image_np[j][i][1]
            #img2[j][i][2]=image_np[j][i][2]
            
            pixelYmax=int(t[0]*boxes[0][0][2])
            pixelYmin=int(t[0]*boxes[0][0][0])
            pixelXmax=int(t[1]*boxes[0][0][3])
            pixelXmin=int(t[1]*boxes[0][0][1])
            img_extract=image_np[pixelYmin:pixelYmax,pixelXmin:pixelXmax]
            #print(pixelYmin,pixelYmax,pixelXmin,pixelXmax)
            img_extract=image_np[int(t[0]*boxes[0][0][2]):int(t[0]*boxes[0][0][0]),int(t[1]*boxes[0][0][3]):int(t[1]*boxes[0][0][1])]
            #plt.imshow(img_extract)
            rouge = img_extract[:,:,0]
            vert = img_extract[:,:,1]
            bleu = img_extract[:,:,2]
            plt.figure(figsize=(15, 7))
            plt.imshow(rouge,cmap='gray')
      

        

/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Attempting to set identical left == right == -0.5 results in singular transformations; automatically expanding.
/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/ipykernel_launcher.py:62: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [26]:
#import cv2 as cv

t=np.shape(image_np)
img2 = np.zeros((1000,2000,3))

for i in range(int(t[1]*boxes[0][0][1]),int(t[1]*boxes[0][0][3])):
    for j in range(int(t[0]*boxes[0][0][0]),int(t[0]*boxes[0][0][2])):
        
        img2[j][i][0]=image_np[j][i][0]
        img2[j][i][1]=image_np[j][i][1]
        img2[j][i][2]=image_np[j][i][2]
    
        
pixelYmax=int(t[0]*boxes[0][0][2])
pixelYmin=int(t[0]*boxes[0][0][0])
pixelXmax=int(t[1]*boxes[0][0][3])
pixelXmin=int(t[1]*boxes[0][0][1])
img_extract=image_np[pixelYmin:pixelYmax,pixelXmin:pixelXmax]
print(pixelYmin,pixelYmax,pixelXmin,pixelXmax)
img_extract=image_np[int(t[0]*boxes[0][0][2]):int(t[0]*boxes[0][0][0]),int(t[1]*boxes[0][0][3]):int(t[1]*boxes[0][0][1])])


#plt.imshow(img2)
rouge = img_extract[:,:,0]
vert = img_extract[:,:,1]
bleu = img_extract[:,:,2]
plt.imshow(rouge,cmap='gray')

        
        

SyntaxError: invalid syntax (<ipython-input-26-a7b175a637eb>, line 20)

In [21]:
rouge = img_extract[:,:,0]
vert = img_extract[:,:,1]
bleu = img_extract[:,:,2]
plt.imshow(rouge,cmap='gray')
        

/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Attempting to set identical left == right == -0.5 results in singular transformations; automatically expanding.
  after removing the cwd from sys.path.
/home/golden/Documents/my_tensorflow/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  after removing the cwd from sys.path.


In [20]:
imBinaire=img_extract
t_im=np.shape(imBinaire)
BW = im2bw(imBinaire,0.5)
plt.imshow(BW)


NameError: name 'im2bw' is not defined

In [12]:
t=np.shape(image_np)
int(t[0]*boxes[0][0][3])


569

In [3]:
!pip install opencv-python 

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 28.2MB 32kB/s eta 0:00:011


In [7]:
!pip install bounding-box
!pip install bbox

In [9]:
import os

#import _init_paths
import cv2
#from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from utils import *

#format du ground truth: nomClasse,xmin,ymin,xmax,ymax

gt_folder="/home/golden/Documents/ground_Truth"
cpt=0
for file in gt_folder:
    with open(file, 'r') as mon_fichier:
        cpt=cpt+1
        content = mon_fichier.read()
        content.split()
        gt_boundingBox_cpt = BoundingBox(
    imageName=file,
    classId=content[0],
    x=content[1],
    y=content[2],
    w=content[3],
    h=content[4],
    typeCoordinates=CoordinatesType.Relative,
    bbType=BBType.GroundTruth,
    format=BBFormat.XYWH,
    imgSize=(353, 500))
        print(gt_boundingBox_cpt)
        print(content[0])
        content.clear()

ModuleNotFoundError: No module named 'BoundingBoxes'